In [74]:
import pandas as pd
import numpy
import json
import statistics as s
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from collections import OrderedDict


In [3]:
header_df = pd.read_json('~/WWData/money_diaries/data/raw_basic_info.json')
diary_df = pd.read_json('~/WWData/money_diaries/data/raw_diaries.json')

In [4]:
header_df = header_df.T

In [5]:
cols = [x for x in header_df.columns if len(x) <= 100 ]
for col in cols:
    print(col)


Occupation
Industry
Age
Location
Salary
Net Worth
Debt
Paycheck Amount (2x/month)
Pronouns
Monthly Expenses
Rent
Health/Dental/Life/Disability Insurance
457b
Pension
HSA
Car
Cell Phone
Spotify
Netflix
Yoga
Lemonade Rental Insurance
Parking
Car insurance
Gas
Savings
Roth IRA
What was your first job and why did you get it?
Did you worry about money growing up?
Do you worry about money now?
Do you or have you ever received passive or inherited income? If yes, please explain.
Loans
Utilities
Internet
Phone
Renter's Insurance
Health Insurance
Streaming
Food Delivery Subscription
Donations
My Salary
My Wife's Salary
My Paycheck
My Wife's Paycheck
Mortgage
Student Loan
Cable/Internet
Car Insurance
Life Insurance
Home Warranty
Cell Phones
Yard Maintenance
Credit Card
My Husband's Salary
My Paycheck (biweekly)
Husband's Paycheck Amount (biweekly)
Loan Payments
Kindle Unlimited
YouTube Premium
Hulu
Google Fi Cell Phone Plan
Car/Renters/Trailer/RV Insurance
Amazon Prime
Paycheck Amount (biweekly)

ClassPass Membership
Credit card
Spotify Student with Hulu
Health/Life/Disability Insurance
MealPal
Health Insurance, Pension Insurance, and Income Tax
Electricity, Gas, and Water
Transport Cards
Deezer
Total
Personal Loan (for credit card debt)
Savings/Investment Accounts
Barre Studio Membership
My Paycheck Amount (2x/month — moonlighting only gets paid 1x/month)
My husband's paycheck (biweekly)
Mortgage + Property Tax
HOA (includes our water)
Estimated Taxes
Electricity + Gas
Cable + Internet
Magazine, Newspaper, Pandora, & Cloud Storage Subscriptions
Car, Condo, Umbrella Insurance
Wine Club
Car Wash Membership
Paycheck Amount (2x/month, one is lower for union dues)
Monthly MBTA Pass
New York Times subscription
Metro Card
Hulu/Netflix/HBO
Lifetime Movie Club
Paycheck Amount (14x/year )
Retirement Investment Account
Cash Savings Account
Utilities (Gas, Electricity, & Water Tax
Home & Life Insurance
Netflix/Spotify/News Subscriptions
Discount Train Card
Bus Fares
Daily Burn Membership


Personal Insurance
Miscellaneous Bills
Magazine Subscriptions
Gym Fee
TheSkimm App
Regular Savings
Family Maintenance
All Other Expenses
Gym & Social Club Membership
T-Mobile International Plan
Paycheck (Once Per Week)
Cable & Internet, Water & Electricity
Spin Package
Children's Orphanage & Catholic Charities
WaPo
Company Stock Purchasing Program
The Washington Post
Health/HSA/Vision/Dental
Solar Power
Dental, Medical & Vision
Indexed Universal Life Insurance
HBOGo, Showtime (+ other channels)
My Paycheck (Every Other Friday)
Water & Heat
Condo
Monthly Medication
Family
IRS Debt
Migraine Prescription
Paycheck (Every week)
Roth
Employer Retirement
Climb Nashville Membership
My Mom's Rent
Magazines
Paycheck (2x/Month)
eTrade
Metra Unlimited
Paycheck (Every Two Weeks)
Crunchyroll Subscription
Other Monthly Expenses
Husband's Photoshop Adobe Package
Evernote Plus Subscription
Paycheck (2x/week)
MyTherapist.com
Health, Vision & Dental
Life Insurance, AD&D, & LTE
Other Expenses
Monthly Savi

In [71]:
df = header_df.loc[:, header_df.isnull().mean() < .8]

In [28]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(cols)

true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [29]:
for i in range(true_k):
    print(f'Cluster {i}'),
    for ind in order_centroids[i, :20]:
        print(f' {terms[ind]}')

Cluster 0
 internet
 membership
 phone
 payment
 gas
 card
 loan
 cable
 pass
 utilities
 car
 credit
 expenses
 donation
 account
 plan
 student
 app
 cell
 electric
Cluster 1
 savings
 account
 emergency
 term
 investments
 joint
 yield
 short
 high
 personal
 college
 travel
 taxes
 accounts
 contribution
 current
 health
 ally
 daughters
 contractor
Cluster 2
 insurance
 dental
 health
 car
 vision
 life
 medical
 renter
 home
 renters
 disability
 auto
 term
 rental
 phone
 jewelry
 mortgage
 property
 payment
 annual
Cluster 3
 water
 trash
 electric
 gas
 electricity
 sewer
 utilities
 garbage
 heat
 power
 sewage
 recycling
 internet
 hot
 cable
 filter
 drinking
 ladwp
 payment
 la
Cluster 4
 paycheck
 month
 2x
 1x
 biweekly
 week
 weekly
 husband
 weeks
 4x
 partner
 job
 combined
 fiancé
 bi
 time
 freelance
 bimonthly
 friday
 wife
Cluster 5
 husband
 401
 salary
 paycheck
 biweekly
 contribution
 gym
 student
 car
 payment
 tax
 phone
 ira
 roth
 weekly
 loans
 insurance


In [151]:
categories = OrderedDict()

categories['income'] = ['paycheck','salary','income']
categories['investments'] = ['401 k','401','401k','roth','ira','savings','investments','retirement']

categories['insurance'] = ['insurance']
categories['health'] = ['hsa','fsa','dental','therapy','medication','disability','vision','medical','health']
categories['home'] =  ['mortgage','renter s insurance','rent','renters','property tax','hoa','housing','home']
categories['utilities'] =  ['internet','wifi','electric','electricity','water','cable','utility','utilities']
categories['car'] = ['auto','fuel','car']
categories['entertainment'] = ['netflix','hulu','hbo','amazon prime','apple','disney','spotify']
categories['debt']  =  ['credit card','student loan','loans','debt']
categories['pet'] = ['cat','dog','pet']



In [152]:
training_data = {}
for col in cols :
    col = re.sub('\W+',' ', col ).lower()
    training_data[col] = None
    for k,v in categories.items():
        for tag in v :
            if re.search(rf"\w?\b{tag}\b\w?",col): 
                training_data[col] = k         
        
    if training_data[col] is None :
        training_data[col] = 'other'


In [153]:
unlabeled_data = { k: v for k,v in training_data.items() if v =='other' }
labeled_data = { k: v for k,v in training_data.items() if v !='other' }

In [154]:
labeled_data

{'salary': 'income',
 'debt': 'debt',
 'paycheck amount 2x month ': 'income',
 'rent': 'home',
 'health dental life disability insurance': 'health',
 'hsa': 'health',
 'car': 'car',
 'spotify': 'entertainment',
 'netflix': 'entertainment',
 'lemonade rental insurance': 'insurance',
 'car insurance': 'car',
 'savings': 'investments',
 'roth ira': 'investments',
 'do you or have you ever received passive or inherited income if yes please explain ': 'income',
 'loans': 'debt',
 'utilities': 'utilities',
 'internet': 'utilities',
 'renter s insurance': 'home',
 'health insurance': 'health',
 'my salary': 'income',
 'my wife s salary': 'income',
 'my paycheck': 'income',
 'my wife s paycheck': 'income',
 'mortgage': 'home',
 'student loan': 'debt',
 'cable internet': 'utilities',
 'life insurance': 'insurance',
 'home warranty': 'home',
 'credit card': 'debt',
 'my husband s salary': 'income',
 'my paycheck biweekly ': 'income',
 'husband s paycheck amount biweekly ': 'income',
 'hulu': 'en